## 🔍 Market Regime Detection
The various decisions and actions for profit-seeking in financial markets is driven largely by how those markets behave. When asset prices have steadily been in an upward trend for the past several months, for example, many would stay invested or even use leverage to capture more gains. On the other hand, when priecs rise along with higher interest rate, as in a couple years ago, perhaps it’s safer to opt for commodity-related investments.\
These distinct patterns of market behavior are known as market regimes, and they often shift because of changing government policy or broader macroeconomic conditions. Consequently, the effectiveness of different investments can also change, with impacts on returns, volatility, and how prices move over time. \
For this reason, it is vital to detect market regime shifts so that traders and investors can stay informed to make the best choice of quantitative strategy and portfolio allocation to better align with current environment and maximize returns.\
Below are some popular techniques and their applications.

1. Strategy Switching
Different strategies work better in different regimes:
Momentum strategies thrive in trending markets.\
Mean-reversion strategies work better in sideways markets.\
Volatility breakout strategies perform well in high-volatility environments.\
Example: Detect a high-volatility regime → switch from mean-reversion to momentum breakout.

2. Position Sizing / Risk Management
Traders may reduce exposure during uncertain regimes or increase size during favorable ones.\
High volatility → smaller positions.\
Stable bull market → larger positions, tighter stop-losses.\

## 📏 Rule-based Method
Rule-based methods rely on explicit thresholds derived from indicators like volatility, momentum, or moving averages. These methods are simple, transparent, and effective for making real-time risk management decisions, such as adjusting position sizes or stop-loss distances.

In [ ]:
# Import and Setup
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Load historical SPY data
data = yf.download("SPY", start="2015-01-01", end="2024-12-31")
data['LogRet'] = np.log(data['Adj Close']).diff()

# Rolling 20-day volatility as risk regime proxy
data['Volatility'] = data['LogRet'].rolling(window=20).std()

# Define regime: High if above median, else low
vol_median = data['Volatility'].median()
data['Risk_Regime'] = np.where(data['Volatility'] > vol_median, 'High Risk', 'Low Risk')

# Example: Adjust position size
data['Position_Size'] = np.where(data['Risk_Regime'] == 'High Risk', 0.5, 1.0)

## 🫧 K-means Clustering
K-means clustering is an unsupervised learning method that groups similar periods in market data based on features like returns, volatility, and momentum. It allows segmenting historical periods into distinct market regimes, making it easier to assign optimal trading strategies.

In [ ]:
# Import and Setup
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

In [ ]:
# Features: 20-day returns, volatility, and momentum
data['Momentum'] = data['Adj Close'].pct_change(20)
features = data[['LogRet', 'Volatility', 'Momentum']].dropna()
scaled = StandardScaler().fit_transform(features)

# Apply K-Means
kmeans = KMeans(n_clusters=3, random_state=42)
data.loc[features.index, 'KMeans_Regime'] = kmeans.fit_predict(scaled)

# Assign strategy per cluster (example logic)
cluster_strategy_map = {0: 'Momentum', 1: 'Mean Reversion', 2: 'Neutral'}
data['Strategy'] = data['KMeans_Regime'].map(cluster_strategy_map)

## 🕸 Hidden Markov Model (HMM)
HMM is a probabilistic model that assumes market regimes are hidden states generating observable returns and infers these latent states and their probabilities. It enables dynamic strategy switching and probabilistic risk adjustments.

on historical price data, classify regimes such as bull or bear markets. 
Fit on log returns and predict the most probable state at each time step opting for trend-following strategies in one state and mean-reversion strategies in another. 
switch strategies.

In [ ]:
# Import and Setup
from hmmlearn.hmm import GaussianHMM

In [ ]:
# Drop NaNs and use returns for HMM
returns = data['LogRet'].dropna().values.reshape(-1, 1)
hmm = GaussianHMM(n_components=2, covariance_type='full', n_iter=1000, random_state=42)
hmm.fit(returns)
hidden_states = hmm.predict(returns)

# Store regime and probability
data.loc[data['LogRet'].dropna().index, 'HMM_State'] = hidden_states
prob_matrix = hmm.predict_proba(returns)
data.loc[data['LogRet'].dropna().index, 'HMM_Prob_State0'] = prob_matrix[:, 0]

# Example logic: Use strategy & risk scaling based on HMM state
data['HMM_Strategy'] = np.where(data['HMM_State'] == 0, 'Trend', 'Reversion')
data['HMM_Position_Size'] = np.where(data['HMM_Prob_State0'] > 0.8, 1.0, 0.5)

## 📈 Visual Example: Strategy vs. Regimes

In [ ]:
plt.figure(figsize=(12, 6))
for i in range(2):
    mask = data['HMM_State'] == i
    plt.plot(data.index[mask], data['Adj Close'][mask], '.', label=f'HMM Regime {i}')
plt.legend()
plt.title('HMM-Inferred Market Regimes')
plt.show()